In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import string


In [61]:

data_letter = pd.read_csv('./data/letters_CG.csv')
data_letter.head()

,Class,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx,Unnamed: 17
0,C,4,10,5,8,2,5,7,7,10,7,6,13,1,8,4,9,NaN
1,C,3,5,4,4,2,4,8,5,7,12,9,11,1,10,2,7,NaN
2,G,5,10,6,7,3,8,7,8,8,6,7,9,2,7,5,10,NaN
3,C,5,9,6,7,4,3,8,6,7,12,10,13,1,9,3,7,NaN
4,G,4,8,5,6,3,6,6,7,6,10,7,12,2,9,4,9,NaN


In [62]:
X = data_letter.drop(['Class', 'Unnamed: 17'], axis = 1)
y= data_letter.Class

In [63]:
#phan tach 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

In [64]:
# xay dung cay quyet dinh 
clf = DecisionTreeClassifier()

clf = clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)


In [65]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9470198675496688
